In [7]:
# Data Processing
# Ethan L. Edmunds, Jan 2025

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.optimize import curve_fit

In [8]:
# Load the data into a pandas Dataframe
file_path = 'Full_ICY_Durham_data'
df = pd.read_csv(file_path)

critical_current_data = []

unique_tesla = df['Tesla'].unique()
unique_angle = df['Angle'].unique()

print(df.head())

   Tesla  Angle   Current   Voltage   Time
0   0.15    0.0 -0.016081  1.833614  0.046
1   0.15    0.0 -0.016235  1.850597  0.092
2   0.15    0.0 -0.016093  1.850638  0.465
3   0.15    0.0 -0.016118  1.853878  0.512
4   0.15    0.0 -0.016013  1.853580  0.558


In [9]:
# Define the distances in mm (converting to metres)
distance = 12.7/1000

# Voltage correction
for tesla in unique_tesla:
    for angle in unique_angle:

        df_filtered = df[(df['Tesla'] == tesla) & (df['Angle'] == angle)]  # Create a filtered dataframe based on unique value of tesla and angle

        if not df_filtered.empty:

            max_current = df_filtered['Current'].max()

            df_filtered_linear = df_filtered[(df_filtered['Current'] < max_current * 0.4)]  # Ensure you filter on df_filtered, not df

            # Apply a linear fit to the filtered data
            coefficients = np.polyfit(df_filtered_linear['Current'], df_filtered_linear['Voltage'], 1)

            # Calculate the fitted voltages using the polynomial fit
            background_voltage = np.polyval(coefficients, df_filtered['Current'])

            # Subtract the fitted voltages from the original 'Voltage' column to get the 'Fixed_Voltage'
            df.loc[(df['Tesla'] == tesla) & (df['Angle'] == angle), 'Fixed_Voltage'] = df_filtered['Voltage'] - background_voltage

# Calculate Electric Field for both distances (E) for both distances
df['Electric Field'] = df['Fixed_Voltage']/distance

# Identify the power law
def power_law(I, I_c, n):

    I = np.abs(I)

    E_c = 100 # Fixed J_c to be assumed value
    return E_c * (I / I_c) ** n

# Finding n exponent for each curve
# Finding n exponent for each curve
for tesla in unique_tesla:
    for angle in unique_angle:

        # Create a filtered dataframe based on unique value of Tesla and Angle
        df_filtered = df[(df['Tesla'] == tesla) & (df['Angle'] == angle)]

        if not df_filtered.empty:
            # Extract current and electric field from the DataFrame
            I_data = df_filtered['Current']  # Current (A)
            E_data = df_filtered['Fixed_Voltage']  # Fixed Voltage used as the Electric Field

            # Check if the data looks reasonable (no NaNs or Inf)
            if np.any(np.isnan(I_data)) or np.any(np.isnan(E_data)) or np.any(np.isinf(I_data)) or np.any(np.isinf(E_data)):
                print(f"Warning: NaN or infinite values in the data for Tesla: {tesla}, Angle: {angle}. Skipping.")
                continue

            # Initial guess for the fitting parameters [E_c, n]
            initial_guess = [40, 20]  # Adjust these guesses if needed

            try:
                # Perform the curve fitting
                popt, pcov = curve_fit(power_law, I_data, E_data, p0=initial_guess)

                # Extract the fitted parameters
                I_c_fitted, n_fitted = popt

                critical_current_data.append([tesla, angle, I_c_fitted])

                # Print the fitted E_c
                print(f"Fitted E_c for Tesla = {tesla}, Angle = {angle}: {I_c_fitted:.2f}, Fitted n: {n_fitted:.2f}")

            except RuntimeError:
                # Handle the case where the curve fitting does not converge
                print(f"Curve fitting did not converge for Tesla = {tesla}, Angle = {angle}. Skipping.")

df_critical_current = pd.DataFrame(columns=['Tesla', 'Angle', 'Critical Current'], data=critical_current_data)


Fitted E_c for Tesla = 0.15, Angle = 0.0: 51.17, Fitted n: 12.80
Fitted E_c for Tesla = 0.15, Angle = 120.0: 57.57, Fitted n: 14.23
Fitted E_c for Tesla = 0.15, Angle = 150.0: 54.40, Fitted n: 13.04
Fitted E_c for Tesla = 0.15, Angle = 40.0: 56.29, Fitted n: 13.11
Fitted E_c for Tesla = 0.15, Angle = 60.0: 61.72, Fitted n: 12.99
Fitted E_c for Tesla = 0.15, Angle = 90.0: 75.78, Fitted n: 17.09
Fitted E_c for Tesla = 0.2, Angle = 0.0: 46.62, Fitted n: 13.47
Fitted E_c for Tesla = 0.3, Angle = 0.0: 41.90, Fitted n: 11.50
Fitted E_c for Tesla = 0.3, Angle = 120.0: 40.68, Fitted n: 11.30
Fitted E_c for Tesla = 0.3, Angle = 150.0: 40.88, Fitted n: 11.25
Fitted E_c for Tesla = 0.3, Angle = 40.0: 44.32, Fitted n: 11.03
Fitted E_c for Tesla = 0.3, Angle = 60.0: 46.16, Fitted n: 10.78
Fitted E_c for Tesla = 0.3, Angle = 90.0: 62.28, Fitted n: 16.78
Fitted E_c for Tesla = 0.45, Angle = 0.0: 37.39, Fitted n: 11.27
Fitted E_c for Tesla = 0.45, Angle = 120.0: 33.03, Fitted n: 9.80
Fitted E_c for Te

In [10]:
print(df.head())
print(df_critical_current.head())

df.to_csv('Full_ICY_Durham_Data_processed', index=0)
df_critical_current.to_csv('critical_current_data', index=0)

   Tesla  Angle   Current   Voltage   Time  Fixed_Voltage  Electric Field
0   0.15    0.0 -0.016081  1.833614  0.046       0.055995        4.409079
1   0.15    0.0 -0.016235  1.850597  0.092       0.072984        5.746763
2   0.15    0.0 -0.016093  1.850638  0.465       0.073020        5.749586
3   0.15    0.0 -0.016118  1.853878  0.512       0.076261        6.004776
4   0.15    0.0 -0.016013  1.853580  0.558       0.075959        5.981012
   Tesla  Angle  Critical Current
0   0.15    0.0         51.170583
1   0.15  120.0         57.567812
2   0.15  150.0         54.395248
3   0.15   40.0         56.289816
4   0.15   60.0         61.715172
